In [12]:
import cv2
import matplotlib.pyplot as plt
from flask import *
from pyngrok import ngrok
import json
import io
import base64
import numpy as np
import requests
from PIL import Image, ImageDraw, ImageFont
from roboflow import Roboflow
import ultralytics
from flask_cors import CORS

In [14]:
port_no = 3030
from flask_cors import CORS

In [13]:
import cv2

def highlight_regions(image_path, results):
    # Assuming 'results' is an ultralytics.engine.results.Results object
    # Get the original image
    image = cv2.imread(image_path)

    # Get the detected boxes in [x1, y1, x2, y2] format
    boxes = results.boxes.xyxy

    # Get the confidence scores for each box
    confidences = results.boxes.conf

    # Get the class labels for each box
    class_labels = results.boxes.cls

    # Iterate over the detected boxes and highlight regions
    for box, conf, cls in zip(boxes, confidences, class_labels):
        x1, y1, x2, y2 = map(int, box)
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
        class_name = results.names[int(cls)]
        label = f"{class_name} ({conf:.2f})"
        
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(image, label, (x1, y1 - 10), font, 0.75, (0, 255, 0), 1, cv2.LINE_AA)

    return image

In [16]:

port_no = 3030
CORS(app, resources={r"/*": {"origins": "*"}})

app = Flask(__name__)

@app.route("/alzhimer", methods=["POST"])
def decode_base64_and_display_alzhimer():
    try:
        data = request.json
        base64_string = data["base64"]
        disease_name = data["disease"]
        
        decoded_data = pybase64.b64decode(base64_string)
        image = Image.open(io.BytesIO(decoded_data))
        temp_file_path = "C:\\Users\\djsma\\Downloads\\temp_image.jpg"
        image.save(temp_file_path)
        
        # Load the trained YOLOv8 model
        model = YOLO('C:/Users/djsma/Downloads/Github_Desktop/Gojo-Backend/runs/detect/train4/weights/best.pt')
        
        # Perform prediction on the uploaded image
        results = model.predict(source=temp_file_path, save=True)
        
        # Assuming 'results' is a list containing a single ultralytics.engine.results.Results object
        result = results[0]
        
        # Highlight regions and save the image
        highlighted_image = highlight_regions(temp_file_path, result)
        cv2.imwrite("C:\\Users\\djsma\\Downloads\\highlighted_image.jpg", highlighted_image)
        
        # Convert the highlighted image to base64
        image_with_rectangle = Image.fromarray(cv2.cvtColor(highlighted_image, cv2.COLOR_BGR2RGB))
        buffered = io.BytesIO()
        image_with_rectangle.save(buffered, format='jpeg')
        final_base64_string = base64.b64encode(buffered.getvalue()).decode()
        
        # Prepare the inference result
        inference = 'This is from the IoT device'
        
        return jsonify({
            "base64": final_base64_string,
            "inference": inference
        })
    except Exception as e:
        print(e)
        return jsonify({
            "status": "Something went wrong"
        })

app.run(port=port_no)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:3030
Press CTRL+C to quit
127.0.0.1 - - [20/Apr/2024 08:47:46] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [20/Apr/2024 08:51:14] "POST /?api_key=MDUmhShkcQTpnD7H6ZtL HTTP/1.1" 404 -
127.0.0.1 - - [20/Apr/2024 08:51:43] "POST /?api_key=MDUmhShkcQTpnD7H6ZtL HTTP/1.1" 404 -
127.0.0.1 - - [20/Apr/2024 08:52:17] "POST /alzhimer?api_key=MDUmhShkcQTpnD7H6ZtL HTTP/1.1" 200 -


415 Unsupported Media Type: Did not attempt to load JSON data because the request Content-Type was not 'application/json'.
